In [1]:
from ccip_terminal import estimate_gas_limit_from_recent_ccip, load_accounts, get_dynamic_gas_fees, ROUTER_MAP, send_ccip_transfer, estimate_dynamic_gas

In [2]:
from dotenv import load_dotenv
import os

# Gas Estimates

In [3]:
PRIVATE_KEY = os.getenv('PRIVATE_KEY')

In [4]:
account_obj = load_accounts('arbitrum')

In [5]:
w3 = account_obj[0]['w3']

In [6]:
account = w3.eth.account.from_key(PRIVATE_KEY)
account_obj={"account": account, "w3": w3}

In [7]:
gas_limit_est = estimate_gas_limit_from_recent_ccip(w3)

In [8]:
gas_limit_est

274569

In [9]:
fees = get_dynamic_gas_fees(w3)

In [10]:
max_fee_per_gas = fees["max_fee_per_gas"]

In [11]:
fees.keys()

dict_keys(['base_fee', 'max_priority_fee', 'max_fee_per_gas', 'gas_price'])

In [12]:
gas_limit_est

274569

In [13]:
etherscan_estimate = ((gas_limit_est * max_fee_per_gas) / 1e18) * 1.25

In [14]:
gas_limit = estimate_dynamic_gas('arbitrum')
gas_estimate = (gas_limit * max_fee_per_gas) / 1e18

In [15]:
gas_limit

405999

In [16]:
estimate = min(etherscan_estimate,gas_estimate)

In [17]:
ROUTER_MAP['ethereum']

'0x80226fc0Ee2b096224EeAc085Bb9a8cba1146f7D'

In [18]:
#Approves Router to Spend Our Token If Necessary

# approve_token_if_needed(TOKEN_CONTRACTS[source_chain], 
#                             ROUTER_MAP[source_chain], 
#                             account_obj, 
#                             threshold=None)

In [19]:
receipt, links, success, message_id = send_ccip_transfer(
    to_address='0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84',
    dest_chain='optimism',
    amount=1,
    source_chain='arbitrum',
    account_obj=account_obj,
    estimate=estimate
)

2025-04-14 20:16:32,570 - INFO - CCIP messageId (pre-send): cf8cae4e57dc905844ae2ee0d349193536c620c12c79280a34be37ca75dfc902


In [20]:
success

True

In [ ]:
links

In [ ]:
# from web3 import Web3

# # Correct tuple-based signature
# sig = "CCIPSendRequested((bytes32,uint64,address,address,address,bytes,bytes,bytes,uint256,uint256,address))"

# # Web3 will compute the topic0 correctly
# topic0 = Web3.keccak(text=sig).hex()
# print(topic0)


003fb4702b81d9460b90fd7d986ba60b240fa8af0d1ebd672660291af318d567
